In [ ]:
import csv 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("data/cardio_datatraining.csv",sep=";")

#   traitement de gender  

In [9]:
df["gender"].unique()

array([1, 2], dtype=int64)

In [10]:
df['gender'] = df['gender'].replace({2: "male", 1: "female"})

In [11]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
68681,98125,20417,female,160,64.0,120,90,3,1,0,0,1,1
19961,28510,22690,male,167,65.0,120,80,3,3,0,0,0,0
11040,15795,22784,female,160,66.0,120,90,1,1,0,0,1,1
27673,39560,22648,female,163,55.0,125,90,3,1,0,0,1,1
22876,32677,21712,female,158,85.0,150,80,3,1,0,0,1,1


In [12]:
df.groupby("gender").mean()

,id,age,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
gender,,,,,,,,,,,,
female,49807.167587,19502.030970,161.351145,72.546524,127.913869,94.458362,1.387198,1.234233,0.018293,0.026294,0.801820,0.496392
male,50137.926095,19376.826795,169.933158,77.223164,130.717221,102.021424,1.334092,1.208348,0.221483,0.109165,0.806363,0.503211


#   traitement de Cholesterol 

In [13]:
df["cholesterol"].unique()

array([3, 1, 2], dtype=int64)

In [14]:
df['cholesterol'] = df['cholesterol'].replace({1: "normal", 2: "above normal",3:"well above normal"})

In [15]:
df['cholesterol'].unique()

array(['well above normal', 'normal', 'above normal'], dtype=object)

#   traitement de glucose

In [16]:
df['gluc'] = df['gluc'].replace({1: "normal", 2: "above normal",3:"well above normal"})

#   modifier l'unitè de age de jours vers ans

In [17]:
df["age"] = df["age"].map(lambda x: x // 365)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49000 entries, 68681 to 15795
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           49000 non-null  int64  
 1   age          49000 non-null  int64  
 2   gender       49000 non-null  object 
 3   height       49000 non-null  int64  
 4   weight       49000 non-null  float64
 5   ap_hi        49000 non-null  int64  
 6   ap_lo        49000 non-null  int64  
 7   cholesterol  49000 non-null  object 
 8   gluc         49000 non-null  object 
 9   smoke        49000 non-null  int64  
 10  alco         49000 non-null  int64  
 11  active       49000 non-null  int64  
 12  cardio       49000 non-null  int64  
dtypes: float64(1), int64(9), object(3)
memory usage: 5.2+ MB


In [19]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
68681,98125,55,female,160,64.0,120,90,well above normal,normal,0,0,1,1
19961,28510,62,male,167,65.0,120,80,well above normal,well above normal,0,0,0,0
11040,15795,62,female,160,66.0,120,90,normal,normal,0,0,1,1
27673,39560,62,female,163,55.0,125,90,well above normal,normal,0,0,1,1
22876,32677,59,female,158,85.0,150,80,well above normal,normal,0,0,1,1


#   traitement des outliers 

In [20]:
df.describe()

,id,age,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio
count,49000.000000,49000.000000,49000.000000,49000.000000,49000.000000,49000.000000,49000.000000,49000.000000,49000.000000,49000.000000
mean,49922.798061,52.812735,164.351347,74.181439,128.893898,97.102347,0.089327,0.055265,0.803408,0.498776
std,28897.596101,6.774974,8.196510,14.377289,158.084444,200.210646,0.285217,0.228500,0.397425,0.500004
min,0.000000,29.000000,55.000000,22.000000,-140.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24916.000000,48.000000,159.000000,65.000000,120.000000,80.000000,0.000000,0.000000,1.000000,0.000000
50%,49938.500000,53.000000,165.000000,72.000000,120.000000,80.000000,0.000000,0.000000,1.000000,0.000000
75%,74936.500000,58.000000,170.000000,82.000000,140.000000,90.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,64.000000,250.000000,200.000000,14020.000000,11000.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
df.columns

Index(['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
       'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')

In [22]:
def valAber(df):
  
    aberrant_counts = {}
    
    for col in df.columns:
        q3 = df[col].quantile(0.75)
        q1 = df[col].quantile(0.25)
        IQR = q3 - q1
        lower_limit = q1 - (IQR * 1.5)
        upper_limit = q3 + (IQR * 1.5)
        num_aberrant = len(df[(df[col] < lower_limit) | (df[col] > upper_limit)])
        aberrant_counts[col] = num_aberrant
    for col, count in aberrant_counts.items():
        print(f"Nombre de valeurs aberrantes dans la colonne '{col}': {count}")


l=['age', 'height', 'weight', 'ap_hi', 'ap_lo','smoke', 'alco', 'active', 'cardio']
valAber(df[l])


Nombre de valeurs aberrantes dans la colonne 'age': 4
Nombre de valeurs aberrantes dans la colonne 'height': 347
Nombre de valeurs aberrantes dans la colonne 'weight': 1243
Nombre de valeurs aberrantes dans la colonne 'ap_hi': 1014
Nombre de valeurs aberrantes dans la colonne 'ap_lo': 3240
Nombre de valeurs aberrantes dans la colonne 'smoke': 4377
Nombre de valeurs aberrantes dans la colonne 'alco': 2708
Nombre de valeurs aberrantes dans la colonne 'active': 9633
Nombre de valeurs aberrantes dans la colonne 'cardio': 0


#   remplacer les vals aberantes par la mediane

In [23]:
def replaceAberrantWithMedian(df, columns):
    for col in columns:
        q3 = df[col].quantile(0.75)
        q1 = df[col].quantile(0.25)
        IQR = q3 - q1
        lower_limit = q1 - (IQR * 1.5)
        upper_limit = q3 + (IQR * 1.5)
        
        df[col] = df[col].clip(lower_limit, upper_limit)


In [24]:
replaceAberrantWithMedian(df,l)

In [25]:
valAber(df[l])

Nombre de valeurs aberrantes dans la colonne 'age': 0
Nombre de valeurs aberrantes dans la colonne 'height': 0
Nombre de valeurs aberrantes dans la colonne 'weight': 0
Nombre de valeurs aberrantes dans la colonne 'ap_hi': 0
Nombre de valeurs aberrantes dans la colonne 'ap_lo': 0
Nombre de valeurs aberrantes dans la colonne 'smoke': 0
Nombre de valeurs aberrantes dans la colonne 'alco': 0
Nombre de valeurs aberrantes dans la colonne 'active': 0
Nombre de valeurs aberrantes dans la colonne 'cardio': 0


#   supprimer id

In [26]:
df.drop(columns=['id'],inplace=True)

In [27]:
df.columns

Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol',
       'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')

In [28]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
68681,55,female,160.0,64.0,120,90,well above normal,normal,0,0,1,1
19961,62,male,167.0,65.0,120,80,well above normal,well above normal,0,0,1,0
11040,62,female,160.0,66.0,120,90,normal,normal,0,0,1,1
27673,62,female,163.0,55.0,125,90,well above normal,normal,0,0,1,1
22876,59,female,158.0,85.0,150,80,well above normal,normal,0,0,1,1
